## 魔术方法
* python自带的方法
* 格式：双下划线包围

#### `__`str`__`
* 当使用 print() / str() / f'{obj}' 时自动调用的方法
* 返回一个人类（用户）可读的，描述对象的字符串
* 没有返回值时会报错
* 与之相对的，__repr__返回开发者明确的描述

In [1]:
class Person:

    def __init__(self, name, color):
        self.name = name
        self.color = color

    def __str__(self):  # 打印对象时输出的信息
        return f'{self.name} is {self.color}'


vct = Person('Victoria', 'white')
print(vct)

Victoria is white


#### `__`new`__`
* __init__是对象的初始化器(Initializer)，给已经存在的对象赋初始值
* __new__是对象的构造器(Constructor)，负责创建对象并返回它
* 不刻意声明__new__的话，python会提供一个默认的__new__方法

* 调用时机：__new__在__init__之前被调用
* 参数：__new__至少接收一个参数cls（代表类class本身），而不是self（因为对象还没被创建）
* 返回值：必须返回一个对象实例（cls的实例），若不返回，__init__不会被调用
* 主要用途：控制不可变类型的实例创建，或实行单例模式和自定义对象创建流程

In [2]:
class Person:

    def __init__(self, name, color):
        self.name = name
        self.color = color
        print('执行init')
    def __new__(cls, *args, **kwargs):
        print('执行new')

vct = Person('Victoria', 'white')
print(vct)
# __new__没有返回对象实例，__init__未被执行，对象创建失败

执行new
None


In [3]:
class Person:

    def __init__(self, name, color):
        self.name = name
        self.color = color
        print('执行init')

    def __new__(cls, *args, **kwargs):
        print('执行new')
        return super().__new__(cls)  # 调用父类的__new__方法
        # return object.__new__(cls)    # 调用根类object的__new__方法，更不灵活，不推荐使用


vct = Person('Victoria', 'white')
print(vct)

执行new
执行init


一个单例模式演示

In [4]:
class Singleton:
    _instance = None

    def __new__(cls):
        if cls._instance is None:
            print("Creating new instance")
            cls._instance = super().__new__(cls)
        else:
            print("Instance already exists, returning it")
        return cls._instance


# 测试
s1 = Singleton()  # 输出: Creating new instance
s2 = Singleton()  # 输出: Instance already exists, returning it
print(s1 is s2)  # 输出: True (它们是同一个对象)

Creating new instance
Instance already exists, returning it
True


#### 案例：双人对战

In [5]:
class Player():

    def __init__(self, name):
        self.name = name
        self.blood = 100
        print(self)

    def stab(self, opponent):
        self.damage(opponent, 'stab', 10)

    def strike(self, opponent):
        self.damage(opponent, 'strike', 15)

    def pill(self):
        if self.blood >= 90:
            print(f'{self.name} recovers {100 - self.blood} HP.')
            self.blood = 100
        else:
            print(f'{self.name} recovers 10 HP.')
            self.blood += 10
        print(self)

    def __str__(self):
        return f'{self.name} has {self.blood} HP.'

    def damage(self, opponent, method, damage):
        actual_damage = min(damage, opponent.blood)
        if opponent.blood <= damage:
            print(f'{self.name} {method}s {opponent.name} for {actual_damage} HP.')
            opponent.blood = 0
            print(opponent)
            print(f'{opponent.name} is dead')
        else:
            print(f'{self.name} {method}s {opponent.name} for {actual_damage} HP.')
            opponent.blood -= damage
            print(opponent)


Tom = Player('Tom')
Jerry = Player('Jerry')
print()

Tom.strike(Jerry)
Tom.strike(Jerry)
Tom.strike(Jerry)
Tom.strike(Jerry)
Tom.strike(Jerry)
Tom.strike(Jerry)
Tom.strike(Jerry)

Tom has 100 HP.
Jerry has 100 HP.

Tom strikes Jerry for 15 HP.
Jerry has 85 HP.
Tom strikes Jerry for 15 HP.
Jerry has 70 HP.
Tom strikes Jerry for 15 HP.
Jerry has 55 HP.
Tom strikes Jerry for 15 HP.
Jerry has 40 HP.
Tom strikes Jerry for 15 HP.
Jerry has 25 HP.
Tom strikes Jerry for 15 HP.
Jerry has 10 HP.
Tom strikes Jerry for 10 HP.
Jerry has 0 HP.
Jerry is dead


#### `__`del`__`
* 析构方法 (Deconstructor) 定义对象在“生命结束”前需执行的清理操作
* 当对象即将被销毁时，python的垃圾回收机制 (Garbage Collector) 可能会调用它
* 程序自然结束时自动调用析构方法
* 特点1：调用时机不确定，由GC决定
* 特点2：并非总是会被调用，如强制退出时（不可靠）

* 在Jupyter Notebook中重新运行一个单元格时，会先清理旧变量再执行新代码，不适合学习__del__
* 更多详见 2.4 Deconstructor